In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 87kB/s 
     |████████████████████████████████| 204kB 20.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=f4c50f3214cd6634500b0905ab9de508d6bd4e419f745236ad2b4ad136c0e48b
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [5]:
spark = SparkSession.builder.appName("Regressao").getOrCreate()

In [9]:
treino = spark.read.format('libsvm').load('/content/sample_linear_regression_data.txt')

In [10]:
treino.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [11]:
modelo = LinearRegression(featuresCol='features', labelCol='label',
                          predictionCol='prediction')

In [12]:
lrModel = modelo.fit(treino)

In [13]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [14]:
lrModel.intercept

0.14228558260358093

In [15]:
treino_summary = lrModel.summary

In [16]:
treino_summary.r2

0.027839179518600154

In [17]:
treino_summary.rootMeanSquaredError

10.16309157133015

In [18]:
dados = spark.read.format('libsvm').load('/content/sample_linear_regression_data.txt')

In [21]:
x, y = dados.randomSplit([0.7,0.3])

In [24]:
x.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
+----------

In [26]:
x.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 349|
|   mean|-0.14808140046621054|
| stddev|  10.386184291150313|
|    min| -28.571478869743427|
|    max|  27.111027963108548|
+-------+--------------------+



In [27]:
y.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                152|
|   mean| 1.1867217676473596|
| stddev| 10.131901214278574|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+



In [28]:
modelo_certo = modelo.fit(x)

In [29]:
previsoes = modelo_certo.evaluate(y)

In [30]:
previsoes.rootMeanSquaredError

10.200912848318023

In [31]:
previsoes.r2

-0.02038207657378166

In [32]:
novos = x.select('features')

In [33]:
novos.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [34]:
previsoes_novas = modelo_certo.transform(novos)

In [35]:
previsoes_novas.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| -2.993480589231265|
|(10,[0,1,2,3,4,5,...| 0.9191689003339636|
|(10,[0,1,2,3,4,5,...| -4.215288032512217|
|(10,[0,1,2,3,4,5,...|-1.6593195789007258|
|(10,[0,1,2,3,4,5,...|-0.6516697463699253|
|(10,[0,1,2,3,4,5,...| 2.2945248109199383|
|(10,[0,1,2,3,4,5,...|-3.7782556442657533|
|(10,[0,1,2,3,4,5,...| 2.3308810161447546|
|(10,[0,1,2,3,4,5,...| 0.7174421792893246|
|(10,[0,1,2,3,4,5,...| 1.0940972189637692|
|(10,[0,1,2,3,4,5,...| 0.3288008589755161|
|(10,[0,1,2,3,4,5,...| 1.4856795286763211|
|(10,[0,1,2,3,4,5,...|-2.3494152971240148|
|(10,[0,1,2,3,4,5,...|-1.6649994581308223|
|(10,[0,1,2,3,4,5,...| -1.629495624806464|
|(10,[0,1,2,3,4,5,...|  1.340775029407834|
|(10,[0,1,2,3,4,5,...| 0.6838649317496015|
|(10,[0,1,2,3,4,5,...|-1.4644187258880257|
|(10,[0,1,2,3,4,5,...|    1.6629939256157|
|(10,[0,1,2,3,4,5,...| 1.4698023017413069|
+----------